##This block is to prepare and process the dataset in this project.
All preprocess datasets are stored in csv files so it could be easily pulled out frequently for later analysis
(1) Dataset1 -- Covid data from yelp downloaded dataset
(2) Dataset2 -- State of restaurants from yelp downloaded dataset
(3) Dataset3 -- covid case and death count from CDC downloaded dataset
(4) Dataset4 -- restaurant reviews from yelpapi for temporary closed restaurants

In [2]:
import pandas as pd
import os
from plotnine import ggplot, aes, geom_point, geom_line, scale_x_datetime
from mizani.breaks import date_breaks

##Change the work directory to the folder 'Team Project'

In [3]:
os.chdir('/Documents/MSIS/Fall 2020/Team Project')
os.getcwd()

'/Documents/MSIS/Fall 2020/Team Project'

##preprocess the dataset1: Covid data from yelp: 'Dataset//yelp_academic_dataset_covid_features.json'

Atttributes in this dataset1:
1.Business_id                                                       -- picked
2.COVID-19-related business highlights
3.Restaurants with delivery/takeout enabled
4.Restaurants partnered with Grubhub
5.Businesses with Call to Action buttons enabled
6.Does the business still have Request A Quote enabled?
7.Has the business created a custom page banner during COVID-19?    -- picked
8.Temporary closures                                                -- picked
9.Virtual Services offered

Issue: 
(1) There are some incorrect data and the format of this json file is not good, so it is necessary to preprosss it.
(2) The most recently dataset from yelp is on 6/10/2020

Attributes picked for this project:
1.Business_id   -- the Identity of restaurants
     (eg. '--8xjda27bazl2')
2.CovidBanner   -- banner shown on the first page of the restaurants' websites 
     (eg. 'Because of the pandemic, we cancelled all in person service, deliveray service is open as normal')
3.TemporaryClosedUntil    -- whether the restaurant is tempoary closed
     (eg. 2020-10-01)

In [5]:
file1 = open('Dataset//yelp_academic_dataset_covid_features.json',"r") 
#i = 0
business_id = ''
CovidBanner = ''
TemporaryClosedUntil = ''
df = pd.DataFrame(columns = ['business_id', 'CovidBanner', 'TemporaryClosedUntil']) 

line = file1.readline()
while line:
    
    #if i % 10000 == 0:
    #    print(type(line))
    #    print(line)
    #   print(i)
    #i+=1
    try:
        business_id = line.split('","highlights":')[0].split('"business_id":"')[1]
        CovidBanner = line.split('","Temporary Closed Until":')[0].split(',"Covid Banner":"')[1]
        TemporaryClosedUntil = line.split('","Virtual Services Offered":')[0].split(',"Temporary Closed Until":"')[1]
    
        df = df.append({'business_id' : business_id, 'CovidBanner' : CovidBanner, 'TemporaryClosedUntil' : TemporaryClosedUntil},  
                    ignore_index = True)
        
    except:
        print('error')
        print(i)
        print(line)
        
    line = file1.readline()
file1.close()
df.head()

,business_id,CovidBanner,TemporaryClosedUntil
0,9kXRUIkwdDtnAPO6tVo51g,FALSE,FALSE
1,H6D5HOTfMjrZt7r1EObZ1g,FALSE,FALSE
2,FYddq7fUtzobZcw4jOJgVA,FALSE,FALSE
3,c75jLTjlgA9q3gImLEGT6w,FALSE,FALSE
4,YfzPiY50h_10Sjlg3mnNWQ,FALSE,FALSE


In [6]:
#dataset1 is stored to a csv file
df.to_csv('YelpCovid.csv', index=False)

##preprocess the dataset2: business data from yelp

Attributes:
There are lots of attributes in this dataset like business profile, user profile, reviews etc. However we only extract business_id, IsOpen and state of the restaurants from this dataset.

Issue: 
(1)The most recently downloaded dataset from yelp is before December 2019, so we can not get the reviews for 2020. 
(2)We assume the state and business_id don't update much in 2020. IsOpen may be updated a lot because of the Covid pandemic, so IsOpen is not used in later analysis.

In [7]:
file1 = open('Dataset//yelp_dataset//yelp_academic_dataset_business.json',"r") 
#i = 0
business_id = ''
IsOpen = ''
State = ''
df = pd.DataFrame(columns = ['business_id', 'IsOpen', 'State']) 

line = file1.readline()
while line: #and i < 10:    
    #if i % 10000 == 0:
        #print(type(line))
        #print(line)
    #    print(i)
    #i+=1
    try:
        business_id = line.split('","name":')[0].split('"business_id":"')[1]
        #print(business_id)
        IsOpen = line.split(',"attributes":')[0].split(',"is_open":')[1]
        #print(IsOpen)
        State = line.split('","postal_code":')[0].split(',"state":"')[1]
        #print(State)
        df = df.append({'business_id' : business_id, 'IsOpen' : IsOpen, 'State' : State},  
                    ignore_index = True)
        
    except:
        print('error')
        print(i)
        print(line)
        
    line = file1.readline()
file1.close()
df.head()

,business_id,IsOpen,State
0,f9NumwFMBDn751xgFiRbNA,1,NC
1,Yzvjg0SayhoZgCljUJRF9Q,1,AZ
2,XNoUzKckATkOD1hP6vghZg,1,QC
3,6OAZjbxqM5ol29BuHsil3w,0,NV
4,51M2Kk903DFYI6gnB5I6SQ,1,AZ


In [9]:
df.to_csv('YelpBusinessState.csv', index=False)

##Combine the above dataset1 and dataset2 into one dataset, and stored it as 'yelpState.csv'

In [10]:
left = pd.read_csv('YelpCovid.csv').set_index('business_id')
right = pd.read_csv('YelpBusinessState.csv').set_index('business_id')
result = left.join(right, how = 'inner')
result.to_csv('yelpState.csv', index=True)
result.head()

,CovidBanner,TemporaryClosedUntil,IsOpen,State
business_id,,,,
--1UhMGODdWsrMastO9DZw,FALSE,FALSE,1,AB
--6MefnULPED_I942VcFNA,FALSE,FALSE,1,ON
--7zmmkVg-IMGaXbuVd0SQ,FALSE,FALSE,1,NC
--8LPVSo5i0Oo61X01sV9A,FALSE,FALSE,1,AZ
--9QQLMTbFzLJ_oT-ON3Xw,FALSE,FALSE,1,AZ


##preprocess the dataset3: covid case and death in US daily updated data

Attributes picked:
1.submission_date
2.state
3.tot_cases
4.new_case
5.tot_death
6.new_death

The preprocessed dataset is stored to 'caseCDC.csv'

In [11]:
fields = ['submission_date', 'state', 'tot_cases', 'new_case','tot_death', 'new_death']
casesCDC = pd.read_csv('Dataset//United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv', usecols=fields) 
casesCDC.head()

,submission_date,state,tot_cases,new_case,tot_death,new_death
0,01/22/2020,CO,0,0,0,0
1,01/23/2020,CO,0,0,0,0
2,01/24/2020,CO,0,0,0,0
3,01/25/2020,CO,0,0,0,0
4,01/26/2020,CO,0,0,0,0


In [12]:
#convert datatype to a proper datatype
caseCDC = casesCDC.astype({'submission_date': 'datetime64[ns]'})   \
                   .astype({'tot_cases': 'int'})   \
                   .astype({'new_case': 'int'})   \
                   .astype({'tot_death': 'int'})   \
                   .astype({'tot_death': 'int'})
caseCDC.to_csv('caseCDC.csv', index=False)

##This is the dataset4: reviews in yelp
As the most recently dataset downloaded from yelp contains data for only 2019 and before, we tried to get the reviews for 2020 through yelpapi.
Issue: 
(1)yelp api could only return at most 3 reviews for every api hit, so it is impossible to get all reviews for 2020. 
(2)We get all the reviews we could get from yelp through the api for all temporary closed restaurants in YelpCovid dataset. 
(3)The 3 reviews returned by yelp api is based on the review quality and user profile, it is not based on the review posted date, so we could get multiple years' review for temporary closed restaurants. 
(4)We store reviews to 'YelpReview_2020.csv'

In [13]:
#pip install Yelpapi
from yelpapi import YelpAPI
#api_key is your key copied from Yelp account: new App
api_key = '_h1ZDN4fbjYfpsC7rKWg7WwPMvGK6zbX5nqFTfBCs0IxMFJCpm-DXtQ_xLn3y5a9DXLNBxF-6r-FvzQmpem9eXhlWuRo2UrppIpdlhDAruH6MasgoehjK0nqvO66X3Yx'
yelp_api = YelpAPI(api_key)


In [14]:
# example hit by yelpapi
search_results = yelp_api.reviews_query(id='-0GU6ghA7z4-xotUZfXl1w')
search_results

{'reviews': [{'id': 'CbmCoUGKXIDSSamWBckB3Q',
   'url': 'https://www.yelp.com/biz/sport-clips-haircuts-of-chandler-chandler-2?adjust_creative=6BbY9J2XrUYEHv0uaK955w&hrid=CbmCoUGKXIDSSamWBckB3Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=6BbY9J2XrUYEHv0uaK955w',
   'text': 'Walk in. Two stylists and one guy waiting. They tell me an hour and 15 wait. Poor staff management and not well managed time at the store. \n\nThis has...',
   'rating': 1,
   'time_created': '2019-10-18 15:48:11',
   'user': {'id': 'z8t25ZIHCXkdHRww5lRdew',
    'profile_url': 'https://www.yelp.com/user_details?userid=z8t25ZIHCXkdHRww5lRdew',
    'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/PYkNbM5Vz9dTpZ54pHDhAg/o.jpg',
    'name': 'John R.'}},
  {'id': 'IBQFWl_Gbe1Odowk6VUTcw',
   'url': 'https://www.yelp.com/biz/sport-clips-haircuts-of-chandler-chandler-2?adjust_creative=6BbY9J2XrUYEHv0uaK955w&hrid=IBQFWl_Gbe1Odowk6VUTcw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews

In [15]:
#get restaurant state and temporary closed data
bs = pd.read_csv('yelpState.csv')
bs.head()  

,business_id,CovidBanner,TemporaryClosedUntil,IsOpen,State
0,--1UhMGODdWsrMastO9DZw,FALSE,FALSE,1,AB
1,--6MefnULPED_I942VcFNA,FALSE,FALSE,1,ON
2,--7zmmkVg-IMGaXbuVd0SQ,FALSE,FALSE,1,NC
3,--8LPVSo5i0Oo61X01sV9A,FALSE,FALSE,1,AZ
4,--9QQLMTbFzLJ_oT-ON3Xw,FALSE,FALSE,1,AZ


In [16]:
#get the reviews for those temporary closed restaurants
i = 0
n = len(bs)
business_id = ''
date = ''
text = ''
df = pd.DataFrame(columns = ['business_id', 'date', 'text']) 

for i in range(0,n):
    if i % 10000 == 0:
        print(i)
    if bs['TemporaryClosedUntil'][i] != 'FALSE':        
        business_id = bs['business_id'][i]
        try:
            search_results = yelp_api.reviews_query(id=business_id)
            for x in search_results.get('reviews'):
                date = x['time_created'][0:10]
                text = x['text']
                df = df.append({'business_id' : business_id, 'date' : date, 'text' : text},  
                        ignore_index = True) 
            #print(date)
            #print(date[0:4])
                #if date[0:4] == '2020':
                #    text = x['text']
                #    df = df.append({'business_id' : business_id, 'date' : date, 'text' : text},  
                 #       ignore_index = True)                
        except:
            print(business_id)           
        
df.head()

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
dt9p8SDbclFb68FDEXPDgw
140000
hZsoH0AHkN2OyXjOAmVQNw
150000
160000
170000
180000
sS7E8O6E1kwsOSmwNsz26g
190000
200000


,business_id,date,text
0,-1m9o3vGRA8IBPNvNqKLmA,2020-11-27,Call early for reservations during the pandemi...
1,-1m9o3vGRA8IBPNvNqKLmA,2020-11-14,easy five stars. sexy dark ambiance. beautiful...
2,-1m9o3vGRA8IBPNvNqKLmA,2020-10-15,Bavette's steakhouse is a true Gem. I loved th...
3,-28hEE6AcAUbJctGwvC9kw,2014-12-28,Things are looking good at the Culinary Pharma...
4,-28hEE6AcAUbJctGwvC9kw,2020-11-28,"Yes, my review of the Culinary pharmacy on Nel..."


In [17]:
df.to_csv('YelpReview_2020.csv', index=False)